In [1]:

import seaborn as sns

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from scipy import sparse
import anndata as ad

In [2]:
# base_folder='/home/jovyan/scratch/datasets/stereoSeq/'
# data_folder=base_folder+'/BGIResearch-2022/'

# analysis_folder='/home/jovyan/scratch/userfolders/sahays/stereoseq_pipeline_sahays/analysis_sahays/'
# os.system('mkdir -p '+analysis_folder)

data_folder = './input/stereopy_demo_data/'
raw_input= data_folder+'SS200000135TL_D1.cellbin.gem' #insert the path to the downloaded image 
df = pd.read_csv(raw_input, sep='\t', comment='#')
df=df[['geneID', 'x', 'y', 'MIDCount']]

In [3]:
df['xy_pos']=df.x.astype(str)+':'+df.y.astype(str)


In [4]:
# create the var data
gene_info=pd.DataFrame(index=df.geneID.unique())
gene_info['gene_index']=range(1, len(gene_info.index.values)+1)
gene_info['gene_index']=gene_info.gene_index.astype(str)
gene_info['gene_name']=gene_info.index
gene_info.index=gene_info.gene_index.values
#gene_info=gene_info[['gene_name']]


In [5]:
# create the obs data

bead_info = pd.DataFrame(index=df.xy_pos.unique())
bead_info['bead_index']=range(1, len(bead_info.index.values)+1)
bead_info['bead_index']=bead_info.bead_index.astype(str)
bead_info['bead_xy_pos']=bead_info.index
bead_info.index=bead_info.bead_index.values
bead_info[['x_coords', 'y_coords']]= bead_info['bead_xy_pos'].str.split(":",expand=True)

In [6]:
df=df.merge(bead_info[['bead_index', 'bead_xy_pos']], left_on='xy_pos', right_on='bead_xy_pos')
df=df.merge(gene_info[['gene_index', 'gene_name']], left_on='geneID', right_on='gene_name')
df=df[['MIDCount', 'gene_index', 'bead_index']]

In [7]:
df.set_index(['gene_index', 'bead_index'], inplace=True)
df.reset_index(inplace=True)
df['bead_index']=df['bead_index'].astype(int) 
df['gene_index']=df['gene_index'].astype(int)

In [8]:
mat = sparse.coo_matrix((df.MIDCount, (df['bead_index'], df['gene_index'])))
csr = mat.tocsr()

In [9]:
adata = ad.AnnData(csr)


/tmp/ipykernel_235736/2225634509.py:1: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(csr)


In [10]:
adata.obs_names = df['bead_index'].drop_duplicates(keep='first').astype(str).values
adata.var_names= df['gene_index'].drop_duplicates(keep='first').astype(str).values

ValueError: Length of passed value for obs_names is 8119736, but this AnnData has shape: (8119737, 22414)

In [ ]:
df['bead_index'].drop_duplicates(keep='first').astype(str).values

In [ ]:
adata

In [ ]:
adata.var=gene_info[['gene_name']]
adata.obs=bead_info[['x_coords', 'y_coords']]

In [ ]:
adata.write(data_folder+'converted_SS200000135TL_D1.h5ad', compression="gzip")

In [ ]:
df.index.names

In [ ]:
coo = sparse.coo_matrix((df["MIDCount"], (bead_index, gene_index)), shape=shape)